# Capital Asset Pricing Model

The relationship between risk and return is clear (see §4), but the specific uses for the diversifiable risk (see §2) for determining the value of a security for an investor is remains opaque. In the following section, we will explore how assets can be priced according to their expected risk/return relationship. This system of pricing is know as the Capital Asset Pricing Model (CAPM). Using our understanding of CAPM, we can build portfolios which cater to the needs of clients who are willing to accept differing levels of risk.

The following notebook represents notes taken from the *Python for Finance: Investment Fundamentals & Data Analytics* Udemy course. All credit for the content of the lecture goes to 365 Cereers.

## <a name="TOC"></a> Table of Contents:
---
1. [CAPM Theory](#Theory)
3. [Beta Theory](#Beta)
  1. [Calculating Beta](#CalBeta)
3. [CAPM Expected Return](#CAPMRet)
4. [Sharpe Ratio Theory](#Sharpe)
  1. [Finding the Sharpe Ratio](#FindSharpe)

## <a name="Theory"></a> [CAPM Theory](#TOC)
---

Under the CAPM landscape, investors are risk-averse, prefer higher returns, and are willing to buy the optimal portfolio for their needs. Realistically, investors are not always so rational, but CAPM operates under this assumption. Sharpe, the economist who invented CAPM, consideres a portfolio which represent alls possible investments throughout the global market place. Such a portfolio contains almost no idiosyncratic risk as it reflects the market perfectly. However, it assumes an asset which has a rate of return without any risk, which some investors would be willing to accept instead of placing investments market-wide. Using this risk-free rate of return, we can find that the Market portfolio investors should accept is the portfolio which represents the intersection of the efficient frontier and the capital market line, which is determined by the risk free rate of return.

<img name="Frontier" src="Figures\CAPM.1.JPG" alt="Drawing" style="width: 400px;" align = "right"/>
<img name="Frontier" src="Figures\CAPM.1.JPG" alt="Drawing" style="width: 400px;" align = "right"/>

Using CAPM, we can find an expected return of a security that is more robust than simply using the historical patterns. We can find this CAPM expectation for a given security using the risk-free rate of return at the time of investment combined with the beta of the security distributed to the return of the market minus the risk free rate of return.

$$
r_i = r_f + \beta_{i} (r_m - r_f)
$$

There is no such thing as a risk free asset, of course, but the closest instrument we can use is the 10-year US government bond. We can use the [US Treasury tables](https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield) to find this value. As we can see from the historical patterns of the 10-year government bond yield, the average rate of return is approximately 2.5%. The difference between the market return and the risk-ree rate of return is known as the *Equity Risk Premium* and it fluctuates between 4.5% and 5.5%. Given this range, we use 5% to approximate the Equity Risk Premium.

Before we can apply this theory, however, we must first understand beta as a measurement of comparative volatility and then we can apply the CAPM formula.

## <a name="Beta"></a> [Beta Theory](#TOC)
---

For securities analysis, *beta* represents the degree to which a given security fluctuates with or against a reference market. For example, when the S&P500 fluctuates, does our security of interest fluctuate to the same degree? To a lesser degree? When the S&P500 fluctuates negatively, does our security fluctuate negatively as well? What about positively?

Beta is calculated using the covariance of the security and the index, divided by the variance of the market..

$$
\beta = \frac{\text{cov}(r_s, r_m)}{\sigma^{2}_{m}}
$$

The resulting beta can be classified in accordance with their risk. Some stocks will have very little relationship to the market, such as Walmart with $\beta = 0.09$. When the market fluctuates, Walmart does not see a significant change in its business cycle because their clients will not suddenly stop buying food. Ford, on the other hand, with $\beta = 1.10$ will be more volitile with respect to the market because consumers will adjust their spending habits by avoiding larger purchases during down markets.

| No Relationship | Direct Defensive | Direct Aggressive |
|-----------------|------------------|-------------------|
| $\beta = 0$     | $0 < \beta < 1$  | $\beta > 1$       |

### <a name="CalBeta"></a> [Calculating Beta](#TOC)
---

Now that we have basic understanding of the beta measurement, let's learn to apply calculate the beta using Python.

In [15]:
# -------- IMPORT LIBRARIES -------- #

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Heavily depreciated
#from pandas_datareader import data as wb

# Finance specific
import quandl
import datetime

# Fix the Yahoo API
import fix_yahoo_finance as yf
yf.pdr_override()

# Set Quandl Key
quandl.ApiConfig.api_key = "ar6m_fz_du7d7C1y_xof"


# -------- DOWNLOAD DATA -------- #

assets = 'PG GE'.split()
start = datetime.datetime(2007,1,1)
end = datetime.date.today()

Data = pd.DataFrame()

for a in assets:
    code = "EOD/{}".format(a)
    Data[a] = quandl.get( code, start_date = start, end_date = end )['Adj_Close']
    
# View head of data
Data.tail(5)

,PG,GE
Date,,
2019-04-09,104.66,9.22
2019-04-10,104.65,9.16
2019-04-11,104.75,9.12
2019-04-12,105.06,9.00
2019-04-15,105.73,8.98


In [2]:
# ---------- STUDY RELATIONSHIPS ---------- #

# Find LROR for each asset
LROR  = np.log(Data / Data.shift(1))
ALROR = LROR.mean() * 250

# Find security covariance with market
cov  = LROR.cov() * 250
MCov = cov.iloc[0, 1]

# Find market variance
MVar = LROR['^GSPC'].var() * 250

# Study mean, covariance, and correlation
print("Average Annualized Return: \n", ALROR)
print("\nCovariance: \n", cov)
print("\nCorrelation: \n", LROR.corr())

Average Annualized Return: 
 PG       0.077370
^GSPC    0.111653
dtype: float64

Covariance: 
              PG     ^GSPC
PG     0.020409  0.010078
^GSPC  0.010078  0.016362

Correlation: 
              PG     ^GSPC
PG     1.000000  0.551507
^GSPC  0.551507  1.000000


We can see that P&G gives us a rate of return of 7.7% and the S&P500 yields 11.2% for the period of study. Additionally, we can see that the correlation between our security and the market is strong at 55%. Using the metrics we calculated, we can find and study the beta.

In [3]:
PG_beta = MCov / MVar

print("P&G Beta: ", round(PG_beta, 3))

P&G Beta:  0.616


Using what we learned in beta theory section, we can see that P&G is defensive against market volatility because its beta is less than 1. Consequently, when the market soars our stock will increase but will not soar with the market, and then the market crashes our stocks value will depreciate but will show resistance to market trends.

## <a name="CAPMRet"></a> [CAPM Expected Return](#TOC)
---

Using the CAPM expected return formula, let's use Python to find a more robust expected return for our security.

In [4]:
# -------- DOWNLOAD DATA -------- #

assets = '^TNX'.split()

# Instantiate dataframe
TNX = pd.DataFrame()

# Aquire data
for a in assets:
    TNX[a] = wb.DataReader(a, data_source = "yahoo", start = dstart, end = dend)['Adj Close']

# Isolate start yield
risk_free = float(TNX.iloc[0])
risk_free

1.9600000381469727

In [5]:
# Find CAPM expected return
PG_CAPM = (risk_free/100) + PG_beta * (0.05)
print("P&G ROR: ", round(PG_CAPM * 100, 4), "%")

P&G ROR:  5.0398 %


So the CAPM considers that P&G should earn approximately 5% returns given its risk.

## <a name="Sharpe"></a> [Sharpe Ratio Theory](#TOC)
---

What we are working towards, in this section, is the idea that investors need to consider how risk ought to affect their outlook for a given security. The Sharpe ratio allows us to consider securities by their risk adjusted returns. In other words, it allows us to consider how much return we should expected for a given risk. We can find this ratio using the quantity difference between the return of an instrument and the risk-free rate of return divided by the standard deviation of the instrument.

$$
\frac{r_i - r_f}{\sigma_i}
$$

This ratio reflects the excess return of a security from the risk-free asset with respect to the risk of the security itself. The Sharpe ratio moves positively with the expected return of the stock and moves negatively with the volatility of that stock. Consequently, we can compare stocks and portfolios against each other on a risk adjusted basis.

### <a name="FindSharpe"></a> [Finding the Sharpe Ratio](#TOC)
---

Using the formula for the Sharpe ratio, we can easily use the statistics we found in the earlier python sections to find our numbers.

In [6]:
# Find Sharpe ratio
PG_sharpe = (PG_CAPM - (risk_free/100)) / (LROR['PG'].std() * 250 ** 0.5)
print("P&G Sharpe: ", round(PG_sharpe * 100, 4), "%")

P&G Sharpe:  21.558 %


Now that we have found our Sharpe ratio for P&G, we can compare this number with other investments to determine which instrument offers the best return for the risk assumed by investors.

*Notes prepared by Austin Dial on 3/20/2019*